In [18]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import glob
import os

In [19]:
# Setting up some dictionaries etc.
team_name_to_id = {'Cardinals': 'ARI', 'Falcons': 'ATL', 'Colts': 'IND', 'Ravens': 'BAL', 'Patriots': 'NE', 'Bills': 'BUF', 'Panthers': 'CAR', 'Bears': 'CHI', 'Bengals': 'CIN', 'Browns': 'CLE', 'Cowboys': 'DAL', 'Broncos': 'DEN', 'Lions': 'DET', 'Packers': 'GB', 'Oilers': 'TEN', 'Texans': 'HOU', 'Jaguars': 'JAX', 'Chiefs': 'KC', 'Raiders': 'LVR', 'Chargers': 'LAC', 'Rams': 'LAR', 'Dolphins': 'MIA', 'Vikings': 'MIN', 'Saints': 'NO', 'Giants': 'NYG', 'Jets': 'NYJ', 'Eagles': 'PHI', 'Steelers': 'PIT', '49ers': 'SF', 'Seahawks': 'SEA', 'Buccaneers': 'TB', 'Titans': 'TEN', 'Commanders': 'WAS'}
long_team_name_to_id = {'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Colts': 'IND', 'Baltimore Ravens': 'BAL', 'Boston Patriots': 'NE', 'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI', 'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL', 'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB', 'Houston Oilers': 'TEN', 'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX', 'Kansas City Chiefs': 'KC', 'Las Vegas Raiders': 'LVR', 'Los Angeles Chargers': 'LAC', 'Los Angeles Raiders': 'LVR', 'Los Angeles Rams': 'LAR', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN', 'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG', 'New York Jets': 'NYJ', 'Oakland Raiders': 'LVR', 'Philadelphia Eagles': 'PHI', 'Phoenix Cardinals': 'ARI', 'Pittsburgh Steelers': 'PIT', 'San Diego Chargers': 'LAC', 'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'St. Louis Cardinals': 'ARI', 'St. Louis Rams': 'LAR', 'Tampa Bay Buccaneers': 'TB', 'Tennessee Oilers': 'TEN', 'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS', 'Washington Football Team': 'WAS', 'Washington Redskins': 'WAS'}
list_of_teams = ['Bills', 'Dolphins', 'Patriots', 'Jets',
'Bengals', 'Steelers', 'Ravens', 'Browns',
'Texans', 'Jaguars', 'Titans', 'Colts',
'Chiefs', 'Chargers', 'Raiders', 'Broncos',
'Cowboys', 'Eagles', 'Commanders', 'Giants',
'Packers', 'Bears', 'Vikings', 'Lions',
'Saints', 'Panthers', 'Falcons', 'Buccaneers',
'49ers', 'Rams', 'Cardinals', 'Seahawks']

In [20]:
# Setting up some functions

# Gather all win probs for a certain week
def win_prob_weekX(weekX_data):
    win_prob_dict = dict()

    for team in list_of_teams:
        win_prob = find_win_prob(team, weekX_data)
        win_prob_dict[team] = win_prob

    return win_prob_dict

# Function that returns the win probability for a certain team
def find_win_prob(team, weekX_data):
    text = weekX_data
    
    # Find index of team name
    start_idx = text.find(team)

    # Find the '%' after that
    percent_idx = text.find('%', start_idx)

    # Get the substring directly before '%'
    win_prob = text[percent_idx-2:percent_idx]    
    return win_prob

# Function that returns the line for a particular team
def find_line(team, weekX_data):
    text = weekX_data

    # Find index of team name
    start_idx = text.find(team)

    # Find the spread signifier after that
    spread_idx = text.find('<div class="spread-stat svelte-1yqqaah">', start_idx)

    spread_html = text[spread_idx:spread_idx+100]
    soup = BeautifulSoup(spread_html, "html.parser")

    value = soup.find("div", class_="spread-stat").text
    return value
    
def lines_weekX(weekX_data):
    line_dict = dict()

    for team in list_of_teams:
        try:
            line = find_line(team, weekX_data)
        except Exception:
            continue

        if line is not None:
            line_dict[team_name_to_id[team]] = line
    
    return line_dict


In [21]:
# Read in html data from The Athletic's Website
all_weeks = {}

# Base path to data folder
data_path = os.path.join("..", "data")

athletic_path = os.path.join(data_path, "The Athletic")
# Regular season data
for filepath in glob.glob(os.path.join(athletic_path, 'week*.html')):
    week_name = os.path.basename(filepath).replace('.html', '')
    with open(filepath, 'r', encoding='utf-8') as f:
        all_weeks[week_name] = f.read()

# Playoff data
for playoff_file in ['Wildcard.html', 'Division.html', 'Conference.html', 'Superbowl.html']:
    key = playoff_file.replace('.html', '')  # 'Wildcard', etc.
    path = os.path.join(athletic_path, playoff_file)
    with open(path, 'r', encoding='utf-8') as f:
        all_weeks[key] = f.read()

In [22]:
# --- Create dictionary of weekly line data ---
all_weeks_line_dict = {week: lines_weekX(html) for week, html in all_weeks.items()}

# --- Read in Spreadspoke CSV data ---
df_interim = pd.read_csv(os.path.join(data_path, 'Spreadspoke', 'spreadspoke_scores.csv'))
df = df_interim[df_interim['schedule_season'] == 2024].copy()

# --- Create new columns for home and away team IDs ---
df['team_home_id'] = df['team_home'].map(long_team_name_to_id)
df['team_away_id'] = df['team_away'].map(long_team_name_to_id)

df['team_underdog_id'] = np.where(
    df['team_favorite_id'] == df['team_home_id'],
    df['team_away_id'],
    df['team_home_id']
)

# Fix typo in original data
df.at[13833,'team_favorite_id'] = 'NYJ'

# --- Create column for lines from The Athletic ---
df['calculated_line'] = None

for week_name, week_data in all_weeks_line_dict.items():
    # For regular weeks, extract the week number (playoffs stay as is)
    if week_name.lower().startswith('week'):
        week_number = week_name.replace("week", "")
    else:
        week_number = week_name  # 'Wildcard', 'Division', etc.

    df.loc[df['schedule_week'] == week_number, 'calculated_line'] = (
        df.loc[df['schedule_week'] == week_number, 'team_favorite_id'].map(week_data)
    )


In [23]:
# Fill in calculated_line. The blanks currently represent games where the Vegas favorite differs from The Athletic favorite
df['calculated_line'] = df['calculated_line'].replace("",np.nan)

for week_name, week_data in all_weeks_line_dict.items():
    week_number = week_name.replace('week', '')

    mask = (df['schedule_week'] == week_number) & (df['calculated_line'].isna())

    df.loc[mask, 'calculated_line'] = (
        df.loc[mask, 'team_underdog_id']
        .map(week_data)
        .str[1:]
    )


In [24]:
# Convert pick-ems to 0
df.loc[df['calculated_line'] == 'PK', 'calculated_line'] = 0
df.loc[df['calculated_line'] == 'K', 'calculated_line'] = 0


# Convert columns to numeric and take the difference
df['spread_favorite'] = df['spread_favorite'].astype("Float32")
df['calculated_line'] = df['calculated_line'].astype("Float32")

# Calculate difference between calculated line and spread
df['diff'] = df['spread_favorite'] - df['calculated_line']

# See if we should take action or not
df['action'] = None

# If diff is negative, we're betting on the underdog
# If diff is positive, we're betting on the favorite
df.loc[df['diff'] < 0, 'action'] = 'bet on underdog'
df.loc[df['diff'] > 0, 'action'] = 'bet on favorite'

# Let's check to see how many games there are where The Athletic's spread does not match the Vegas spread
n = df[abs(df['diff']) > 0].shape[0]
print(f"Number of games with mismatched spread: {n}")

Number of games with mismatched spread: 239


In [25]:

# Step 1: Determine if favorite is home or away
df["favorite_score"] = np.where(df["team_favorite_id"] == df["team_home_id"],
                                df["score_home"],
                                df["score_away"])

df["underdog_score"] = np.where(df["team_underdog_id"] == df["team_home_id"],
                                df["score_home"],
                                df["score_away"])

# Step 2: Compute margin
df["margin_fav_underdog"] = df["underdog_score"] - df["favorite_score"]


df['result_column'] = None
df['modeled_v_actual_vegas'] = abs(df['spread_favorite'] - df['margin_fav_underdog'])
df['modeled_v_actual_nyt'] = abs(df['calculated_line'] - df['margin_fav_underdog'])
df.at[14085, 'modeled_v_actual_nyt'] = 0




In [26]:
# Check out in total how far off the Vegas lines and The Athletic's lines were
avg_residual_athletic = df['modeled_v_actual_nyt'].mean()
avg_residual_vegas = df['modeled_v_actual_vegas'].mean()

median_residual_athletic = df['modeled_v_actual_nyt'].median()
median_residual_nyt = df['modeled_v_actual_vegas'].median()

print("*** Averages ***")
print(f"The Athletic: {avg_residual_athletic: .2f}")
print(f"Vegas: {avg_residual_vegas: .2f}")
print("\n*** Medians ***")
print(f"The Athletic: {median_residual_athletic: .2f}")
print(f"Vegas: {median_residual_nyt: .2f}")

*** Averages ***
The Athletic:  9.75
Vegas:  9.66

*** Medians ***
The Athletic:  7.50
Vegas:  7.00


In [27]:
# Main function that calculates wins and losses. I added the "fade" argument after discovering that betting based on my initial idea was not a good strategy
# My intial idea was to bet on the underdog in cases where The Athletic projected a smaller line than Vegas and to bet on the favorite in cases where The Athletic projected a bigger line than Vegas
# Flipping the "fade" argument to true flips those actions

def bet_results(df: pd.DataFrame, fade: bool=False, threshold: float=0.0, mode: str="both") -> pd.DataFrame:
    
    df = df.copy()
    df['result_column'] = None
    diff = df['diff']

    # Step 1: Fade picks if requested
    if fade:
        df.loc[df['action'] == 'bet on underdog', 'action'] = 'bet on favorite_'
        df.loc[df['action'] == 'bet on favorite', 'action'] = 'bet on underdog_'

    # Step 2: Threshold mask
    if mode == "positive":
        threshold_mask = diff >= threshold
    elif mode == "negative":
        threshold_mask = diff <= -threshold
    elif mode == "both":
        threshold_mask = (diff >= threshold) | (diff <= -threshold)
    else:
        raise ValueError("mode must be 'positive', 'negative', or 'both'")

    # Apply results only where threshold condition is satisfied
    # Underdog bets
    for und_key in ['bet on underdog', 'bet on underdog_']:
        mask = (df['action'] == und_key) & threshold_mask
        df.loc[mask & (df['margin_fav_underdog'] > df['spread_favorite']), 'result_column'] = 100
        df.loc[mask & (df['margin_fav_underdog'] < df['spread_favorite']), 'result_column'] = -110
        df.loc[mask & (df['margin_fav_underdog'] == df['spread_favorite']), 'result_column'] = 0

    # Favorite bets
    for fav_key in ['bet on favorite', 'bet on favorite_']:
        mask = (df['action'] == fav_key) & threshold_mask
        df.loc[mask & (df['margin_fav_underdog'] < df['spread_favorite']), 'result_column'] = 100
        df.loc[mask & (df['margin_fav_underdog'] > df['spread_favorite']), 'result_column'] = -110
        df.loc[mask & (df['margin_fav_underdog'] == df['spread_favorite']), 'result_column'] = 0

    return df



In [28]:
# Function to calculate and print ROI of a particular strategy
def print_roi(df: pd.DataFrame, fade: bool=False, threshold: float = 0.0, mode: str="both"):
    
    results = bet_results(df=df, fade=fade, threshold=threshold, mode=mode)
    win_loss = results['result_column'].sum()

    if mode == "both":
        investment = sum(results['result_column'].notna()) * 110
    elif mode == "positive":
        temp_df = results[results['diff'] >= threshold]
        investment = sum(temp_df['result_column'].notna()) * 110
        win_loss = temp_df['result_column'].sum()
    else:
        temp_df = results[results['diff'] <= threshold]
        investment = sum(temp_df['result_column'].notna()) * 110
        win_loss = temp_df['result_column'].sum()
    roi = win_loss / investment

    print(f"Number of bets placed: {investment/110:,.0f}")
    print(f"Total investment: {investment}")
    print(f"Total winnings/loss: {win_loss}")
    print(f"ROI: {roi:.2%}")

In [29]:
# How well would the initial betting strategy work?

initial = print_roi(df, fade=False, threshold=0, mode="both")

# Double check this works
test = bet_results(df=df, fade=False, threshold=0, mode="both")
test_sum = test['result_column'].sum()
print(f"\nTest sum is {test_sum}")

Number of bets placed: 239
Total investment: 26290
Total winnings/loss: -1360
ROI: -5.17%

Test sum is -1360


In [30]:
# How about employing the complete opposite strategy?

opposite = print_roi(df, fade=True, threshold=0, mode="both")

Number of bets placed: 239
Total investment: 26290
Total winnings/loss: -940
ROI: -3.58%


In [31]:
# Gut-check to see what our ROI would be if we won 50% of our bets
f_f = 100 + -110
investment = 110*2
roi = f_f / investment
print(f"ROI when winning 50% of bets: {roi:.2%}")

ROI when winning 50% of bets: -4.55%


In [32]:
# What about games where The Athletic has the favorite favored by more (2 points more, in this case)?
test = print_roi(df=df, fade=False, threshold=2, mode="positive")


Number of bets placed: 18
Total investment: 1980
Total winnings/loss: 540
ROI: 27.27%


In [33]:
# Creeate a table for games where The Athletic has the favorite favored by more
rows = []
for diff in [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]:
    temp_df = bet_results(df=df, fade=False, threshold=diff, mode="positive")
    profit = temp_df[temp_df['diff'] >= diff]['result_column'].sum()
    num_games = (temp_df['diff'] >= diff).sum()
    investment = num_games * 110
    roi = (profit / investment) * 100 if investment !=0 else None
    
    rows.append({
    "Spread Difference": diff,
    "Number of Games": num_games,
    "Profit": profit,
    "Investment": investment,
    "Return": roi
    })

table = pd.DataFrame(rows)

styled_table = (
    table.style
    .format({"Return": "{:.2f}%",
             "Investment": "${:,.0f}",
             "Spread Difference": "\u2265{:.1f}",
             "Profit": lambda x: f"-${abs(x):,.0f}" if x < 0 else f"${x:,.0f}"
    })
    .set_table_styles([{"selector": "th", "props": [("background-color", "#f2f2f2"), ("font-weight", "bold")]}])
    .hide(axis="index")
)

import dataframe_image as dfi
dfi.export(
    styled_table,
    "../return_table.png",
    table_conversion="chrome"
)

In [34]:
# Create a table for games where The Athletic's spread had the favorite favored by less
rows = []
for diff in [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]:
    diff = diff * -1
    temp_df = bet_results(df=df, fade=False, threshold=diff, mode="negative")
    profit = temp_df[temp_df['diff'] <= diff]['result_column'].sum()
    num_games = (temp_df['diff'] <= diff).sum()
    investment = num_games * 110
    roi = (profit / investment) * 100 if investment !=0 else None
    
    rows.append({
    "Spread Difference": diff,
    "Number of Games": num_games,
    "Profit": profit,
    "Investment": investment,
    "Return": roi
    })

table = pd.DataFrame(rows)

styled_table = (
    table.style
    .format({"Return": "{:.2f}%",
             "Investment": "${:,.0f}",
             "Spread Difference": "\u2264{:.1f}",
             "Profit": lambda x: f"-${abs(x):,.0f}" if x < 0 else f"${x:,.0f}"
    })
    .set_table_styles([{"selector": "th", "props": [("background-color", "#f2f2f2"), ("font-weight", "bold")]}])
    .hide(axis="index")
)

import dataframe_image as dfi
dfi.export(
    styled_table,
    "../return_table2.png",
    table_conversion="chrome"
)